# 不均衡データのサンプリング

参考文献;\
https://ohke.hateblo.jp/entry/2017/08/18/230000

公式ドキュメント:\
https://imbalanced-learn.readthedocs.io/en/stable/api.html

  - Under Sampling：負例を減らす
  - Over Sampling：正例を増やす

## 必要なライブラリのインポート

In [29]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

## サンプルデータの取得

今回はkaggleで提供されているCredit Card Fraud Detectionデータセットを使います。

ヨーロッパの人が持つカードで、2013年9月の2日間の取引を記録したデータセットです。
1取引1レコードとなっており、各レコードには不正利用か否かを表す値(1ならば不正利用)を持っていますが、当然ながらほとんどが0で、極めて不均衡なデータセットとなっています。 また、個人情報に関わるため、タイムスタンプと金額以外の項目が主成分分析(および標準化)済みとなっていることも特徴です。

In [2]:
# CSVファイルをDataFrameへロード
original_df = pd.read_csv('creditcard.csv')

In [3]:
original_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


不正利用かどうかは'Class'列に入っており、1ならば不正利用です。

Classが0のサンプル数が284,315に対して、1のサンプル数は492と、不均衡な分布となっている。

In [4]:
original_df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

## ロードした状態のままのデータで分析モデルを作成する。

ランダムフォレスト(決定木は100個)で学習・テストさせる。

### 説明変数と目的変数の抽出

In [5]:
X = original_df.loc[:, 'V1':'Amount']
y = original_df.loc[:, 'Class':]

### 学習データとテストデータの分離

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

### ランダムフォレストによる学習

In [7]:
rfc = RandomForestClassifier(random_state=0, n_estimators=100)
rfc.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### 検証

In [10]:
print('Train score: {:.4f}'.format(rfc.score(X_train, y_train)))
print('Test score: {:.4f}'.format(rfc.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, rfc.predict(X_test))))
print('f1 score: {:.4f}'.format(f1_score(y_test, rfc.predict(X_test))))

Train score: 1.0000
Test score: 0.9995
Confusion matrix:
[[142151     10]
 [    66    177]]
f1 score: 0.8233


結果、テストデータのスコアは99.95%という高い値となりました。

しかし、もともとかなり偏りがあって全て0と答えるだけでも99.82%が達成されることを鑑みると、決して良い結果ではない。 

混合行列を見ても、不正利用であると誤って判定(偽陽性、False Positive: FP)しているサンプル数が10に対して、

不正利用ではないと誤って判定(偽陰性、False Negative: FN)しているサンプル数が66になっている。

つまり、`不正利用ではないと判定されやすくなっている`ことがわかる。

## imblanced-learn

偏りが大きいデータセットに対して、多少の偽陽性が増えたとしても、偽陰性を減らしたい場合がある。

ここでは、誤検出が少し増えても不正利用を検出したい、というニーズで話を進める。

そうした場合に、学習に使われる`陽性サンプルの割合`を増やすことで偽陰性を減らす、という方法がある。

割合を操作するには、大きく括ると以下の3つのやり方がある。
  - 陰性サンプルを減らす(under-sampling)
  - 陽性サンプルを増やす(over-sampling)
  - 上記両方を行う

## under-sampling

### 不正利用のサンプル数をカウント

不正利用しているサンプルが1、不正利用していないサンプルが0になっている。

In [12]:
# 不正利用のサンプル数をカウント
positive_count_train = y_train['Class'].sum()
print('positive count: {}'.format(positive_count_train))

positive count: 249


### ランダムにunder-sampling

In [18]:
rus = RandomUnderSampler(sampling_strategy={0:positive_count_train*9, 1:positive_count_train}, random_state=0)
X_train_resampled, y_train_resampled = rus.fit_sample(X_train, y_train)
print('X_train_resampled.shape: {}, y_train_resampled: {}'.format(X_train_resampled.shape, y_train_resampled.shape))
print('y_train_resample:\n{}'.format(y_train_resampled['Class'].value_counts()))

X_train_resampled.shape: (2490, 29), y_train_resampled: (2490, 1)
y_train_resample:
0    2241
1     249
Name: Class, dtype: int64


### under-sampling後にランダムフォレストで分析モデルの生成

In [20]:
# ランダムフォレストにて学習
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train_resampled, y_train_resampled)

y_pred = rfc.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


### 検証

In [21]:
print('Train score: {:.4f}'.format(rfc.score(X_train_resampled, y_train_resampled)))
print('Test score: {:.4f}'.format(rfc.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, y_pred)))
print('f1 score: {:.4f}'.format(f1_score(y_test, y_pred)))

Train score: 1.0000
Test score: 0.9986
Confusion matrix:
[[142007    154]
 [    40    203]]
f1 score: 0.6767


同じパラメータでランダムフォレストを使って学習・テストすると、FNが66から43まで減っていることがわかります。 

一方でFPは10から157となっており、誤検出が大幅に増えている。

## over-sampling

under-samplingの場合とは逆で、陽性サンプルを増やすことで、不正利用の割合を10%にする。

### ランダムにover-sampling

In [26]:
ros = RandomOverSampler(sampling_strategy={0: X_train.shape[0], 1: X_train.shape[0]//9}, random_state=0)
X_train_resampled, y_train_resampled = ros.fit_sample(X_train, y_train)
print('X_train_resampled.shape: {}, y_train_resampled: {}'.format(X_train_resampled.shape, y_train_resampled.shape))
print('y_train_resample:\n{}'.format(y_train_resampled['Class'].value_counts()))

X_train_resampled.shape: (158225, 29), y_train_resampled: (158225, 1)
y_train_resample:
0    142403
1     15822
Name: Class, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/imblearn/utils/_validation.py:324: UserWarning: After over-sampling, the number of samples (142403) in class 0 will be larger than the number of samples in the majority class (class #0 -> 142154)
  n_samples_majority,


### ランダムフォレストにて学習

In [27]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train_resampled, y_train_resampled)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### 検証

In [28]:
print('Train score: {:.4f}'.format(rfc.score(X_train_resampled, y_train_resampled)))
print('Test score: {:.4f}'.format(rfc.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, rfc.predict(X_test))))
print('f1 score: {:.4f}'.format(f1_score(y_test, rfc.predict(X_test))))

Train score: 1.0000
Test score: 0.9995
Confusion matrix:
[[142151     10]
 [    60    183]]
f1 score: 0.8394


ランダムフォレストで学習・テストさせると、FNは66から60まで減っていますが、under-samplingと比較すると効果は薄くなっています。 

同じ不正利用のサンプルが増えるだけでは、学習モデルに与える影響は小さくとどまる傾向にあるようです。

### SMOTE: under-samplingとover-samplingの組み合わせ

imbalanced-learnではランダム以外にもいくつかover-samplingの実装が用意されており、その1つにSMOTE(Synthetic Minority Over-sampling Technique)がある。

SMOTEは、今あるサンプルをコピーするのではなく、異なる値を持つサンプルを新たに生成することで増やす方法である。（陽性サンプル間を結ぶ直線上に新しいサンプルをプロットするイメージのような感じである）

まずは、under-samplingで不正利用の割合を1%にまで増やす。\
その後、SMOTEで不正利用のサンプルを10倍にすることで不正利用の割合を約10%となるようにする。

### under-samplingで不正利用の割合を1%まで増やす

In [32]:
positive_count_train = y_train['Class'].sum()
rus = RandomUnderSampler(sampling_strategy={0:positive_count_train*99, 1:positive_count_train}, random_state=0)
X_train_undersampled, y_train_undersampled = rus.fit_sample(X_train, y_train)
print('y_train_undersample:\n{}'.format(y_train_undersampled['Class'].value_counts()))

y_train_undersample:
0    24651
1      249
Name: Class, dtype: int64


### SMOTEで不正利用の割合を10%まで増やす

In [37]:
smote = SMOTE(sampling_strategy={0:positive_count_train*99, 1:positive_count_train*10}, random_state=0)
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_undersampled, y_train_undersampled)
print('y_train_resample:\n{}'.format(y_train_resampled['Class'].value_counts()))

y_train_resample:
0    24651
1     2490
Name: Class, dtype: int64


### ランダムフォレストにて学習

In [38]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train_resampled, y_train_resampled)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### 検証

In [39]:
print('Train score: {:.4f}'.format(rfc.score(X_train_resampled, y_train_resampled)))
print('Test score: {:.4f}'.format(rfc.score(X_test, y_test)))
print('Confusion matrix:\n{}'.format(confusion_matrix(y_test, rfc.predict(X_test))))
print('f1 score: {:.4f}'.format(f1_score(y_test, rfc.predict(X_test))))

Train score: 1.0000
Test score: 0.9992
Confusion matrix:
[[142086     75]
 [    41    202]]
f1 score: 0.7769


学習・テストの結果、FNをRandomUnderSampler単体の場合に近い41まで減らしている一方で、FPの増加は75まで抑えている。

under-samplingだけでは、不正利用のサンプルに寄った過学習を起こしていたが、SMOTEと組み合わせることで軽減できることを確認できた。